# Working With Relational Databases

## Creating a Database Engine

A **database engine** is the underlying component that a DBMS uses to create, read, update, and delete (CRUD) data from a database

The term ***database engine*** is frequently used interchangeably with ***database server*** or ***database management system***.

*In this example we will use SQLite and SQLAlchemy*

In [1]:
from sqlalchemy import create_engine
# Required parameter: A string that indicates the type of database you're connecting to, and the name of the database
engine = create_engine('sqlite:///Northwind.sqlite')
table_names = engine.table_names()
print(table_names)

# Note: Before we can query the database, we need to connect to the engine

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


## Querying relational databases in Python


**Workflow of SQL querying**

1. Import packages and functions
2. Create the database engine
3. Connect to the engine
4. Query the database
5. Save query results to a DataFrame
6. Close the connection

### Example: SQL Query with SQLAlchemy

In [2]:
# 1) Import packages and functions
from sqlalchemy import create_engine
import pandas as pd

# 2) Create the database engine
engine = create_engine('sqlite:///Northwind.sqlite')

# 3) Connect to the engine
con = engine.connect()

# 4) Query Database - Returns a SQLAlchemy results object
results = con.execute("SELECT * FROM Employee")

# 5) Save the query results into a Dataframe (Turn SQLAlchemy results object into a DataFrame)
df = pd.DataFrame(results.fetchall())
df.columns = results.keys()
df.head(2)

Id LastName FirstName                  Title TitleOfCourtesy   BirthDate  \
0   1  Davolio     Nancy   Sales Representative             Ms.  1980-12-08   
1   2   Fuller    Andrew  Vice President, Sales             Dr.  1984-02-19   

     HireDate                     Address     City         Region PostalCode  \
0  2024-05-01  507 - 20th Ave. E. Apt. 2A  Seattle  North America      98122   
1  2024-08-14          908 W. Capital Way   Tacoma  North America      98401   

  Country       HomePhone Extension Photo  \
0     USA  (206) 555-9857      5467  None   
1     USA  (206) 555-9482      3457  None   

                                               Notes  ReportsTo  \
0  Education includes a BA in psychology from Col...        2.0   
1  Andrew received his BTS commercial in 1974 and...        NaN   

                              PhotoPath  
0  http://accweb/emmployees/davolio.bmp  
1   http://accweb/emmployees/fuller.bmp

In [3]:
# 6) Close the connection
con.close()

### Example: Using the Context Manager

When using the Context Manager, we do not need to explicitly close the connection

In [4]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///Northwind.sqlite')

# Using Context Manager
with engine.connect() as con:
    results = con.execute('''
        SELECT * FROM Employee
    ''')
    df = pd.DataFrame(results.fetchmany(size=3))
    df.columns = results.keys()
    print(df.head())

   Id   LastName FirstName                  Title TitleOfCourtesy   BirthDate  \
0   1    Davolio     Nancy   Sales Representative             Ms.  1980-12-08   
1   2     Fuller    Andrew  Vice President, Sales             Dr.  1984-02-19   
2   3  Leverling     Janet   Sales Representative             Ms.  1995-08-30   

     HireDate                     Address      City         Region PostalCode  \
0  2024-05-01  507 - 20th Ave. E. Apt. 2A   Seattle  North America      98122   
1  2024-08-14          908 W. Capital Way    Tacoma  North America      98401   
2  2024-04-01          722 Moss Bay Blvd.  Kirkland  North America      98033   

  Country       HomePhone Extension Photo  \
0     USA  (206) 555-9857      5467  None   
1     USA  (206) 555-9482      3457  None   
2     USA  (206) 555-3412      3355  None   

                                               Notes  ReportsTo  \
0  Education includes a BA in psychology from Col...        2.0   
1  Andrew received his BTS commerci

### Querying Relational Databases directly with pandas

**The pandas way to query**: It is possible to query the database in just 1 line, using pandas.read_sql_query()

In [5]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///Northwind.sqlite')
df = pd.read_sql_query('SELECT * FROM Employee', engine)
df.head()

Id   LastName FirstName                  Title TitleOfCourtesy   BirthDate  \
0   1    Davolio     Nancy   Sales Representative             Ms.  1980-12-08   
1   2     Fuller    Andrew  Vice President, Sales             Dr.  1984-02-19   
2   3  Leverling     Janet   Sales Representative             Ms.  1995-08-30   
3   4    Peacock  Margaret   Sales Representative            Mrs.  1969-09-19   
4   5   Buchanan    Steven          Sales Manager             Mr.  1987-03-04   

     HireDate                     Address      City         Region PostalCode  \
0  2024-05-01  507 - 20th Ave. E. Apt. 2A   Seattle  North America      98122   
1  2024-08-14          908 W. Capital Way    Tacoma  North America      98401   
2  2024-04-01          722 Moss Bay Blvd.  Kirkland  North America      98033   
3  2025-05-03        4110 Old Redmond Rd.   Redmond  North America      98052   
4  2025-10-17             14 Garrett Hill    London  British Isles    SW1 8JR   

  Country       HomePhone Extension Photo  \
0     USA  (206) 555-9857      5467  None   
1     USA  (206) 555-9482      3457  None   
2     USA  (206) 555-3412      3355  None   
3     USA  (206) 555-8122      5176  None   
4      UK   (71) 555-4848      3453  None   

                                               Notes  ReportsTo  \
0  Education includes a BA in psychology from Col...        2.0   
1  Andrew received his BTS commercial in 1974 and...        NaN   
2  Janet has a BS degree in chemistry from Boston...        2.0   
3  Margaret holds a BA in English literature from...        2.0   
4  Steven Buchanan graduated from St. Andrews Uni...        2.0   

                                PhotoPath  
0    http://accweb/emmployees/davolio.bmp  
1     http://accweb/emmployees/fuller.bmp  
2  http://accweb/emmployees/leverling.bmp  
3    http://accweb/emmployees/peacock.bmp  
4   http://accweb/emmployees/buchanan.bmp

### Advanced querying: Exploiting Table Relationships


In [6]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///Northwind.sqlite')
# Query displaying Employee's FirstName and their associated Territory
df = pd.read_sql_query('''
    SELECT Employee.FirstName, Territory.TerritoryDescription
    FROM EmployeeTerritory
    INNER JOIN Employee
    ON Employee.Id = EmployeeTerritory.EmployeeId
    INNER JOIN Territory
    on Territory.Id = EmployeeTerritory.TerritoryId
''', engine)
df.head()

FirstName TerritoryDescription
0     Nancy               Wilton
1     Nancy               Neward
2    Andrew             Westboro
3    Andrew              Bedford
4    Andrew            Georgetow